# Example 3: Extended usage and output diagnostics

In this notebook, we demonstrate extended usage of `pyvbmc`. We will:
- take a brief look at `pyvbmc`'s diagnostic output,
- show you how to continue optimization starting from the results of a previous run, and
- show you how to save the results of optimization to disk.

This notebook is Part 3 of a series of notebooks in which we present various example usages for VBMC with the `pyvbmc` package.

In [14]:
import numpy as np
import scipy.stats as scs
from scipy.optimize import minimize
from pyvbmc.vbmc import VBMC
from pyvbmc.formatting import format_dict
import dill

## 1. Model definition and setup

For demonstration purposes, we will run VBMC with a restricted budget of function evaluations, insufficient to achieve convergence. Then we will inspect the output diagnostics, and resume optimization.

We use a higher-dimensional analogue of the same toy target function in Example 1, a broad [Rosenbrock's banana function](https://en.wikipedia.org/wiki/Rosenbrock_function) in $D = 4$.

In [2]:
D = 4  # A four-dimensional problem
prior_mu = np.zeros(D)
prior_var = 3 * np.ones(D)


def log_prior(theta):
    """Multivariate normal prior on theta."""
    cov = np.diag(prior_var)
    return scs.multivariate_normal(prior_mu, cov).logpdf(theta)

The likelihood function of your model will in general depend on the observed data. This data can be fixed as a global variable, as we did directly above for `prior_mu` and `prior_var`. It can also be defined by a default second argument: to `pyvbmc` there is no difference so long as the function can be called with only a single argument (the parameters `theta`):

In [3]:
def log_likelihood(theta, data=np.ones(D)):
    """D-dimensional Rosenbrock's banana function."""
    # In this simple demo the data just translates the parameters:
    theta = np.atleast_2d(theta)
    theta = theta + data

    x, y = theta[:, :-1], theta[:, 1:]
    return -np.sum((x**2 - y) ** 2 + (x - 1) ** 2 / 100, axis=1)


def log_joint(theta, data=np.ones(D)):
    return log_likelihood(theta, data) + log_prior(theta)

In [4]:
LB = np.full(D, -np.inf)  # Lower bounds
UB = np.full(D, np.inf)  # Upper bounds
PLB = np.full(D, prior_mu - np.sqrt(prior_var))  # Plausible lower bounds
PUB = np.full(D, prior_mu + np.sqrt(prior_var))  # Plausible upper bounds

In a typical inference scenario, we recommend starting from a "good" point (i.e. one near the mode). We can run a  quick preliminary optimization (though a more extensive optimization would not harm).

In [5]:
np.random.seed(42)
x0 = np.random.uniform(PLB, PUB)  # Random point inside plausible box
x0 = minimize(lambda t: -log_joint(t), x0).x

In [6]:
# Limit number of function evaluations
options = {
    "max_fun_evals": 10 * D,
}
# We can specify either the log-joint, or the log-likelihood and log-prior.
# In other words, the following lines are equivalent:
vbmc = VBMC(
    log_likelihood,
    x0,
    LB,
    UB,
    PLB,
    PUB,
    user_options=options,
    log_prior=log_prior,
)
# vbmc = VBMC(
#     log_joint,
#     x0, LB, UB, PLB, PUB, user_options=options,
# )

Reshaping x0 to row vector.
Reshaping lower bounds to (1, 4).
Reshaping upper bounds to (1, 4).
Reshaping plausible lower bounds to (1, 4).
Reshaping plausible upper bounds to (1, 4).


(`pyvbmc` expects the bounds to be `(1, D)` row vectors, and the initial point(s) to be of shape `(n, D)`, but it will accept and re-shape vectors of shape `(D,)` as well.)

## 2. Running the model and checking convergence diagnostics

In [7]:
vp, elbo, elbo_sd, success_flag, info = vbmc.optimize()

Beginning variational optimization assuming EXACT observations of the log-joint.
 Iteration  f-count    Mean[ELBO]    Std[ELBO]    sKL-iter[q]   K[q]  Convergence  Action
     0         10          -7.47         4.29    193392.56        2        inf     start warm-up
     1         15          -2.59         2.04         6.38        2        inf     
     2         20          -2.16         5.54         8.18        2        156     
     3         25          -3.81         1.89         4.15        2         81     
     4         30          -4.70         0.16         2.83        2       50.6     
     5         35          -4.61         0.45         1.00        2       18.5     
     6         40          -4.65         0.76         0.85        2       16.8     
   inf         40          -4.26         0.40         0.19       50       18.5     finalize
Inference terminated: reached maximum number of function evaluations options.max_fun_evals.
Estimated ELBO: -4.264 +/-0.396.
Caution: Re

`pyvbmc` is warning us that convergence is doubtful. We can look at the output for more information and diagnostics.

In [8]:
print(success_flag)

False


`False` means that `pyvbmc` has not converged to a stable solution within the given number of function evaluations.

In [15]:
print(format_dict(info))

{
    'function': '<function VBMC.__init__.<locals>.log_joint at 0x7f57ef2c0a60>',
    'problemtype': 'unconstrained',
    'iterations': 6,
    'funccount': 40,
    'bestiter': 5,
    'trainsetsize': 35,
    'components': 50,
    'rindex': 18.492418206913968,
    'convergencestatus': 'no',
    'overhead': nan,
    'rngstate': 'rng',
    'algorithm': 'Variational Bayesian Monte Carlo',
    'version': '0.0.1',
    'message': 'Inference terminated: reached maximum number of function evaluations options.max_fun_evals.',
    'elbo': -4.264123086039281,
    'elbo_sd': 0.3956311379215213,
}


In the `info` dictionary:
- the `convergencestatus` field says 'no' (probable lack of convergence);
- the reliability index `rindex` is 3.68, (should be less than 1).
Our diagnostics tell us that this run has not converged, suggesting to increase the budget.

Note that convergence to a solution does not mean that it is a _good_ solution. You should always check the returned variational posteriors, and ideally should compare across multiple runs of `pyvbmc`.

## 3. Continuing optimization from a previous result

We can continue running `pyvbmc` where we left off by calling the `optimize()` method again. But first we will change the function evaluation budget to a more realistic value.

In [10]:
# This verbose syntax is required to avoid unintentionally modifying
# options after initialization:
vbmc.options.__setitem__("max_fun_evals", 50 * (D + 2), force=True)
_, _, _, _, _ = vbmc.optimize()

Beginning variational optimization assuming EXACT observations of the log-joint.
 Iteration  f-count    Mean[ELBO]    Std[ELBO]    sKL-iter[q]   K[q]  Convergence  Action
     0         50          -5.17         0.13         0.39        2        inf     
     1         55          -4.77         0.03         0.30        2        inf     
     2         60          -4.67         0.02         0.03        2      0.947     
     3         65          -4.63         0.01         0.01        3      0.261     
     4         70          -4.59         0.01         0.01        4      0.345     
     5         75          -4.55         0.00         0.01        5      0.255     
     6         80          -4.47         0.00         0.04        6      0.904     rotoscale, undo rotoscale
     7         85          -4.45         0.00         0.00        9      0.133     
     8         90          -4.40         0.00         0.00       12      0.234     
     9         95          -4.36         0.00   

## 4. Saving results

We can also save the `VBMC` instance to disk and reload it later, in order to continue optimization, sample from the posterior, check the results, etc.

In [11]:
with open("vbmc_test_save.pkl", "wb") as f:
    dill.dump(vbmc, f)

In [12]:
with open("vbmc_test_save.pkl", "rb") as f:
    vbmc_2 = dill.load(f)
samples, components = vbmc_2.vp.sample(5)
# `samples` are samples drawn from the variational posteror.
# `components` are the index of the mixture components each
#  sample was drawn from.
print(samples)
print(components)

[[-1.4910157  -1.6156586  -0.45491049  0.28147664]
 [-0.46736824 -1.15133437 -0.5428148  -1.03736289]
 [-1.94537561 -0.5196604   0.2577638  -0.08569637]
 [-0.28644483 -0.76665024 -0.41776842  1.15168657]
 [-0.37318163 -0.16395405 -0.62844519 -1.13576615]]
[3 1 0 2 1]


## 5. Conclusions

In this notebook, we have given a brief overview of `pyvbmc`'s output diagnostics, and shown how to save, load, and resume optimization. 

In the next notebook, we will illustrate running `pyvbmc` multiple times in order to validate the results.

## Example 3: full code

The following cell includes in a single place all the code used in Example 3, without the extra fluff.

In [13]:
assert False  # skip this cell

import numpy as np
import scipy.stats as scs
from scipy.optimize import minimize
from pyvbmc.vbmc import VBMC
import dill


D = 4  # A four-dimensional problem
prior_mu = np.zeros(D)
prior_var = 3 * np.ones(D)


def log_prior(theta):
    """Multivariate normal prior on theta."""
    cov = np.diag(prior_var)
    return scs.multivariate_normal(prior_mu, cov).logpdf(theta)


def log_likelihood(theta, data=np.ones(D)):
    """D-dimensional Rosenbrock's banana function."""
    # In this simple demo the data just translates the parameters:
    theta = np.atleast_2d(theta)
    theta = theta + data

    x, y = theta[:, :-1], theta[:, 1:]
    return -np.sum((x**2 - y) ** 2 + (x - 1) ** 2 / 100, axis=1)


def log_joint(theta, data=np.ones(D)):
    return log_likelihood(theta, data) + log_prior(theta)


LB = np.full(D, -np.inf)  # Lower bounds
UB = np.full(D, np.inf)  # Upper bounds
PLB = np.full(D, prior_mu - np.sqrt(prior_var))  # Plausible lower bounds
PUB = np.full(D, prior_mu + np.sqrt(prior_var))  # Plausible upper bounds


np.random.seed(42)
x0 = np.random.uniform(PLB, PUB)  # Random point inside plausible box
x0 = minimize(lambda t: -log_joint(t), x0).x


# Limit number of function evaluations
options = {
    "max_fun_evals": 10 * D,
}
# We can specify either the log-joint, or the log-likelihood and log-prior.
# In other words, the following lines are equivalent:
vbmc = VBMC(
    log_likelihood,
    x0,
    LB,
    UB,
    PLB,
    PUB,
    user_options=options,
    log_prior=log_prior,
)
# vbmc = VBMC(
#     log_joint,
#     x0, LB, UB, PLB, PUB, user_options=options,
# )


vp, elbo, elbo_sd, success_flag, info = vbmc.optimize()


print(success_flag)


print(info)


# This verbose syntax is required to avoid unintentionally modifying
# options after initialization:
vbmc.options.__setitem__("max_fun_evals", 50 * (D + 2), force=True)
_, _, _, _, _ = vbmc.optimize()


with open("vbmc_test_save.pkl", "wb") as f:
    dill.dump(vbmc, f)


with open("vbmc_test_save.pkl", "rb") as f:
    vbmc_2 = dill.load(f)
samples, components = vbmc_2.vp.sample(5)
# `samples` are samples drawn from the variational posteror.
# `components` are the index of the mixture components each
#  sample was drawn from.
print(samples)
print(components)

AssertionError: 

## Acknowledgments

Work on the `pyvbmc` package was funded by the [Finnish Center for Artificial Intelligence FCAI](https://fcai.fi/).